# Define functions

In [ ]:
def getCounts(Activity, n,what):
#input: Activity Log and n top used resources 
#output: print count for the top n resources per resource category

      ResourcesFiltered = Activity[ Activity['Event name'] == 'Course module viewed' ]

      SummaryR = pd.DataFrame({'count' :ResourcesFiltered.groupby(['Component', 'Event context']).size()}).reset_index()

      SummaryR.sort_values(['Component','count','Event context'],ascending=False)
      
      SummaryR = SummaryR.set_index('Event context')
      
      if (what == 'T'):
              return pd.DataFrame(SummaryR.groupby(['Component'])['count'].nlargest(n))
      else:
              return pd.DataFrame(SummaryR.groupby(['Component'])['count'].nsmallest(n))

      
      
      
def getTermSummary():
      ViewData = Activity.loc[Activity['Event name'].isin(['Course viewed','Course module viewed'])]

      Summary = pd.DataFrame({'count' : ViewData.groupby( ['User full name','Component','Event name'] ).size()}).reset_index()

      # change from long to wide

      activities_term_wide = Summary.pivot('User full name','Component','count')

      activities_term_wide = activities_term_wide.fillna(0)
      
      activities_term_wide = pd.DataFrame(activities_term_wide).reset_index()
      
      activities_term_wide = activities_term_wide.add_prefix('lms_Count_')
      
      activities_term_wide = activities_term_wide.rename(columns={'lms_Count_User full name':StdCol})

      return activities_term_wide



def getWeeklySummary():
      ViewData = Activity.loc[Activity['Event name'].isin(['Course viewed','Course module viewed'])]
      wgp = ViewData.groupby(['User full name', pd.Grouper(key='Time', freq='W-MON')])['Click'].sum().reset_index().sort_values('User full name')
      wgp = pd.DataFrame(wgp)
      wgp.reset_index()
      wgp.sort_values(['User full name','Time'])
      cols = ['User full name','Time']

      index  = pd.MultiIndex.from_product([wgp[col].unique() for col in cols],
                                         names=cols)

      result = wgp.set_index(['User full name','Time']).reindex(index, fill_value=0).reset_index()
#       del result['Event name'] 

#       result["ComponentWeek"] =  result["Component"] + '_' + result["Time"].map(str)

      # change from long to wide
      activities_week_wide = result.pivot('User full name','Time','Click')

      #activities_week_wide = activities_week_wide.fillna(0)
      activities_week_wide['Student'] = activities_week_wide.index

      return activities_week_wide
    
    
def getDailySummary():
      ViewData = Activity.loc[Activity['Event name'].isin(['Course viewed','Course module viewed'])]
      dgp = ViewData.groupby(['User full name', pd.Grouper(key='Time', freq='D')])['Click'].sum().reset_index().sort_values('User full name')
      dgp = pd.DataFrame(dgp)
      dgp.reset_index()
      #type(wgp)
      dgp.sort_values(['User full name','Time'])

      cols = ['User full name','Time']

      index  = pd.MultiIndex.from_product([dgp[col].unique() for col in cols],
                                         names=cols)

      result = dgp.set_index(['User full name','Time']).reindex(index, fill_value=0).reset_index()


      # change from long to wide
      activities_day_wide = result.pivot('User full name','Time','Click')


      #activities_week_wide = activities_week_wide.fillna(0)
      activities_day_wide['Student'] = activities_day_wide.index

      return activities_day_wide
    
    
def fixTimeColumn():
      if not (np.issubdtype(Activity['Time'].dtype, np.datetime64)):
        for item, frame in Activity.iterrows():
          frame['Time'] = datetime.datetime.strptime(frame['Time'], "%d %b, %H:%M")
        Activity['Time'] = pd.to_datetime(Activity['Time'])
      StartYear = pd.to_numeric(StartDate.year)
      EndYear   = pd.to_numeric(EndDate.year)
      if '10' in Activity['Time'].dt.strftime("%m").unique().tolist():
        Activity['Time'] = Activity['Time'].apply(lambda x:x.replace(year=EndYear) if x.month < 8 else x.replace(year=StartYear) )  
      else :
        Activity['Time'] = Activity['Time'].apply(lambda x:x.replace(year=StartYear))
      return       pd.DataFrame(Activity.dtypes,columns=['Data Type'])    

def findPattern():
      Activity['Click']=1
      Activity['Pattern_Description']=''
      for item, row in Activity.iterrows(): 
        Activity.loc[item, "Pattern_Description"] = re.sub("(?:id|submission|Turnitin) '.*?'", " '#'",row['Description'] )
      return pd.DataFrame( Activity.groupby(['Component','Event name', 'Pattern_Description']).size() )
      #return pd.DataFrame({'count' : Activity.groupby(['Component','Event name', 'Pattern_Description']).size()}).reset_index()   
    
def SubmissionAnalysis():
      SAA =  pd.DataFrame(Activity[(Activity['Event name'].isin(['Add Submission','The user has accepted the statement of the submission.']) ) ].groupby(['User full name','Event context'])['Time'].max())

      SAA.Time = SAA.Time.dt.date

      SAAS = pd.DataFrame({'Count' : SAA.groupby(['Event context','Time']).size()} ).reset_index()

      # df2  = pd.DataFrame([['cc','2017-12-03',4],['cc','2017-12-04',11],['cc','2017-12-02',11]],columns=['Event context','Time','Count'])

      # SAAS = pd.concat([df2,SAAS])

      SAAS = SAAS.sort_values(['Event context','Time']).reset_index()

      Submission = pd.DataFrame( SAAS.loc[SAAS.reset_index().groupby(['Event context'])['Count'].idxmax()] )

      Submission = Submission[['Event context','Time']]

      Submission.rename(index=str, columns={"Time": "deadline"},inplace=True)

      Submission['deadline'] = pd.to_datetime(Submission['deadline'])

      Submission = Submission.sort_values(['deadline']).reset_index(drop=True)
      print('Submission deadline as calculated are:')
      display (pd.DataFrame(Submission) )
      print ('------------------------------------------')

      # Assuming that all assignmnet can be find based on the 'Event name'

      SA = Activity[(Activity['Event name'].isin(['Add Submission','The user has accepted the statement of the submission.']) ) ]

      SA = SA.merge(Submission, how='left', on='Event context')

      SA.sort_values(['User full name','Event context'], inplace=True)

      SA.drop_duplicates(['User full name','Event context'], inplace=True)

      SA = pd.DataFrame(SA, columns= list(SA) + ['lms_Assignment{}_delay'.format(x) for x in range(1, len(Submission)+1)])

      SA.Time = SA.Time.dt.date

      SA.deadline = SA.deadline.dt.date

      i=1 
      for itemx, framex in Submission.iterrows():
        diff = 'lms_diff'+ str(i) + '_delay'
        for itemy, framey in SA.iterrows():
          if framex['Event context'] == framey['Event context']:
            SA.loc[itemy, diff] = (  framey['Time'] - framey['deadline'] ).days
          #       print (framey['deadline'])
          #       print (framey['Time'])
          #       print (framey['deadline'] - framey['Time'])
          #       print ( framey['deadline'] - framey['Time']  ) 
          #       print ('************')
        i=i+1

      SA[StdCol] = SA['User full name']

      filter_col = [col for col in SA if col.startswith('lms_Assignment')]
      #filter_col =  + filter_col 

      filter_col.insert(0, StdCol)

      SA = SA[filter_col]

      SA = pd.DataFrame(SA.groupby(StdCol).sum()).reset_index()
      
#       SA = Grades[[StdCol]].merge(SA, how='left', on=StdCol)

#       SA ['ASC'] = ASSC.count(axis=1) - 2

      return SA

def accessDaysCount():
      AFA = pd.DataFrame(Activity[['User full name','Time']])
      AFA.Time = AFA.Time.dt.date
      AFA = pd.DataFrame({'lms_ActiveDaysCount' :AFA.groupby('User full name').Time.nunique()}).reset_index()
      AFA.rename(columns={'User full name': StdCol}, inplace=True)
      return AFA
        
def maxGap():
      AFA = pd.DataFrame(Activity[['User full name','Time']])
      AFA = AFA.sort_values(by=['User full name','Time']).drop_duplicates()
      AFA['lms_LInactivityPeriod'] = (AFA.groupby(['User full name'], group_keys=False).apply(lambda g: g['Time'].diff().dt.total_seconds().replace(0, np.nan).ffill()))
      GBD = pd.DataFrame(AFA.groupby(['User full name'])['lms_LInactivityPeriod'].max()).reset_index()
      GBD.rename(columns={'User full name': StdCol}, inplace=True)
      return GBD


def firstlastGap():
      AFA = pd.DataFrame(Activity[['User full name','Time']])
      AFA.Time = AFA.Time.dt.date
      FLG = pd.DataFrame(   AFA.groupby(['User full name'])['Time'].apply(lambda x: x.max() - x.min()) ).reset_index()
      FLG.rename(columns={'User full name': StdCol}, inplace=True)
      FLG['lms_AccessDuration'] = FLG['Time'].dt.days
      FLG = FLG[[StdCol,'lms_AccessDuration']]
      return FLG
    
def resourceView():

    ViewData = Activity.loc[Activity['Event name'].isin(['Course module viewed'])]

    SummaryR = pd.DataFrame({'count' :ViewData.groupby(['Event context', 'User full name']).size()}).reset_index()

    SummaryR.sort_values(['Event context','count','User full name'])

    counter = pd.DataFrame( {'lms_RescourceViewCount' : SummaryR.groupby(['User full name']).size()}).reset_index()

    counter.rename(columns={'User full name': StdCol}, inplace=True)                         

    return counter;

def WeeklyConsistency():
    ViewData = Activity.groupby(['User full name', pd.Grouper(key='Time', freq='W-MON')])['Click'].sum().reset_index().sort_values('User full name')
    SummaryR = pd.DataFrame(ViewData)
    SummaryR.reset_index()
    SummaryR.sort_values(['User full name','Time'])
    groupedData = pd.DataFrame({'lms_ActiveWeekCount' :SummaryR.groupby(['User full name']).size()}).reset_index()
#     SummaryR.sort_values(['lms_ActiveWeekCount','User full name'])
    groupedData.rename(columns={'User full name': StdCol}, inplace=True)                         
    return groupedData;


